In [76]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
import pyodbc

In [77]:
local_ = r"C:\Users\zengsh\OneDrive - Westfund Ltd\Documents\Westfund_"
input_folder = r"\\prdeqs01\QlikData\joins_his\input"
output_folder = r"\\prdeqs01\QlikData\joins_his\output"

In [78]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=prdsql05.westfund.com.au;'
    'DATABASE=BRONZE;'
    'Trusted_Connection=yes;'
)

In [79]:
memship_table = "memship"
memship_snapshot = "memship_20260130"

In [80]:
query = f"""
WITH NewJoins AS (
    SELECT m.membership_id,
           m.effective_join_date
    FROM BRONZE.dbo.{memship_table} m
    LEFT JOIN BRONZE.dbo.{memship_snapshot} snap 
        ON m.membership_id = snap.membership_id
    WHERE snap.membership_id IS NULL
      AND m.memship_status = 'A'
),
LatestCover AS (
    SELECT
        membership_id,
        description AS cover_description,
        ROW_NUMBER() OVER (PARTITION BY membership_id ORDER BY cover_version DESC) AS rn
    FROM BRONZE.dbo.cover
    WHERE status_flag = 'A'
),
LatestPromo AS (
    SELECT
        main_ref AS person_id,
        promotion_id,
        sales_channel_id,
        operator,
        agency_group_id,
        ROW_NUMBER() OVER (PARTITION BY main_ref ORDER BY status_date DESC) AS rn
    FROM BRONZE.dbo.promotion_reference
    WHERE status_date <= GETDATE()
),
LatestMembership AS (
    SELECT
        pm.person_id,
        pm.membership_id,
        pm.relationship,
        ROW_NUMBER() OVER (PARTITION BY pm.person_id ORDER BY pm.membership_id DESC) AS rn
    FROM BRONZE.dbo.person_membership pm
    LEFT JOIN BRONZE.dbo.memship m ON pm.membership_id = m.membership_id
)
SELECT
    nj.membership_id,
    nj.effective_join_date,
    lc.cover_description,
    lm.person_id,
    lp.promotion_id,
    p.description AS promotion_description,
    lp.sales_channel_id,
    ps.description AS sales_channel_description,
    o.first_name + ' ' + o.surname AS Operator,
    COALESCE(g.description, 'No Agency') AS Agent
FROM NewJoins nj
LEFT JOIN LatestCover lc 
    ON nj.membership_id = lc.membership_id 
    AND lc.rn = 1
LEFT JOIN LatestMembership lm 
    ON nj.membership_id = lm.membership_id 
    AND lm.rn = 1 
    AND lm.relationship = 1
LEFT JOIN LatestPromo lp 
    ON lm.person_id = lp.person_id 
    AND lp.rn = 1
LEFT JOIN BRONZE.dbo.promotion p 
    ON lp.promotion_id = p.promotion_id
LEFT JOIN BRONZE.dbo.promotion_sales_channel ps 
    ON lp.sales_channel_id = ps.sales_channel_id
LEFT JOIN BRONZE.dbo.operator o 
    ON lp.operator = o.oper_name
LEFT JOIN BRONZE.dbo.grouping g 
    ON lp.agency_group_id = g.group_id;
"""

df_join = pd.read_sql(query, conn)
len(df_join)

C:\Users\zengsh\AppData\Local\Temp\ipykernel_37180\1560960787.py:71: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_join = pd.read_sql(query, conn)


296

In [81]:
date_suffix = memship_snapshot[-8:]
month_year = (datetime.strptime(date_suffix, "%Y%m%d") + relativedelta(months=1)).strftime("%b %Y")

join_file_name = f"df_join_details_{month_year.replace(' ', '_')}.xlsx"
join_file_path = os.path.join(output_folder, join_file_name)
os.makedirs(output_folder, exist_ok=True)
df_join.to_excel(join_file_path, index=False)
print(f"Saved to {join_file_path}")

Saved to \\prdeqs01\QlikData\joins_his\output\df_join_details_Feb_2026.xlsx


In [82]:
date_suffix = memship_snapshot[-8:]
month_year = (datetime.strptime(date_suffix, "%Y%m%d") + relativedelta(months=1)).strftime("%b %Y")

df_phone = (
    df_join[df_join['sales_channel_description'] == 'Phone']
    .groupby('Operator')
    .size()
    .reset_index(name='Joins')
)
df_phone.insert(0, 'MonthYear', month_year)
df_phone

,MonthYear,Operator,Joins
0,Feb 2026,Amanda Howdin,3
1,Feb 2026,Ashmeeta Prakash,5
2,Feb 2026,Charmaine Findlay,8
3,Feb 2026,Daniel Ryan,7
4,Feb 2026,James Devey,1
5,Feb 2026,Louise Walker,5
6,Feb 2026,Marcia Smythe,1
7,Feb 2026,Mehdi Afrashteh,3
8,Feb 2026,Natalie Henschell,2
9,Feb 2026,Pam Riik,16


In [83]:
file_name = f"joins_sales_{month_year.replace(' ', '_')}.xlsx"
file_path = os.path.join(input_folder, file_name)
os.makedirs(input_folder, exist_ok=True)
df_phone.to_excel(file_path, index=False)
print(f"Saved to {file_path}")

Saved to \\prdeqs01\QlikData\joins_his\input\joins_sales_Feb_2026.xlsx


In [84]:
excel_files = [f for f in os.listdir(input_folder) if f.endswith(('.xlsx', '.xls'))]
dfs = [pd.read_excel(os.path.join(input_folder, f)) for f in excel_files]
joins_sales_his_overall = pd.concat(dfs, ignore_index=True)
print(f"Loaded {len(excel_files)} files, {len(joins_sales_his_overall)} rows total")
joins_sales_his_overall

Loaded 2 files, 5677 rows total


,MonthYear,Operator,Joins,Joins Avg Premium,Total Annual Premium - Joins
0,Feb 2026,Amanda Howdin,3,NaN,NaN
1,Feb 2026,Ashmeeta Prakash,5,NaN,NaN
2,Feb 2026,Charmaine Findlay,8,NaN,NaN
3,Feb 2026,Daniel Ryan,7,NaN,NaN
4,Feb 2026,James Devey,1,NaN,NaN
...,...,...,...,...,...
5672,Sep 2025,Zach Mann,0,-,0.0
5673,Oct 2025,Zach Mann,0,-,0.0
5674,Nov 2025,Zach Mann,0,-,0.0
5675,Dec 2025,Zach Mann,0,-,0.0


In [85]:
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "joins_sales_his_overall.xlsx")
joins_sales_his_overall.to_excel(output_path, index=False)
print(f"Saved to {output_path}")

Saved to \\prdeqs01\QlikData\joins_his\output\joins_sales_his_overall.xlsx
